In [ ]:
!pip install git+https://github.com/suno-ai/bark.git

  Cloning https://github.com/suno-ai/bark.git to /tmp/pip-req-build-v1ay8sbj
  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark.git /tmp/pip-req-build-v1ay8sbj
  Resolved https://github.com/suno-ai/bark.git to commit 773624d26db84278a55aacae9a16d7b25fbccab8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 21.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.8 MB/s eta 0:00:00
  Created wheel for suno-bark: filename=suno_bark-0.0.1a0-py3-none-any.whl size=25674

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


from IPython.display import Audio
import nltk  # we'll use this to split into sentences
import numpy as np

from bark.generation import (
    generate_text_semantic,
    preload_models,
)
from bark.api import semantic_to_waveform
from bark import generate_audio, SAMPLE_RATE

In [ ]:
preload_models()

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" to /root/.cache/torch/hub/checkpoints/encodec_24khz-d7cc33bc.th
100%|██████████| 88.9M/88.9M [00:01<00:00, 80.4MB/s]


# Simple Long-Form Generation
We split longer text into sentences using `nltk` and generate the sentences one by one.

In [ ]:
file_path = "Task2.txt"
with open(file_path, "r", encoding="utf-8") as file:
    text = file.read()
script=text.replace("\n", " ").strip()

In [ ]:
script

'Summary: Judgment on Complaint under Section 138 of the Negotiable Instruments Act Introduction This text discusses a judgment from the Supreme Court of India regarding a complaint filed under Section 138 of the Negotiable Instruments Act. The case involves a dispute over a cheque issued by the respondent, which was returned due to insufficient funds. The Trial Court initially dismissed the complaint, but the Supreme Court upheld it, finding that the cheque was indeed issued by the respondent. Key Points The complaint was dismissed initially due to contradictions in the evidence regarding the number of apple cartons and the amount owed. The High Court established that a cheque carries a presumption of consideration unless proven otherwise. The burden of proof is on the accused to rebut the presumption of consideration by providing evidence or circumstances to show that no debt existed. The court discusses the presumption of debt or liability under Section 139 of the Act and states tha

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
sentences = nltk.sent_tokenize(script)

In [ ]:
SPEAKER = "v2/en_speaker_6"
silence = np.zeros(int(0.25 * SAMPLE_RATE))  # quarter second of silence

pieces = []
for sentence in sentences:
    audio_array = generate_audio(sentence, history_prompt=SPEAKER)
    pieces += [audio_array, silence.copy()]


 75%|███████▌  | 577/768 [01:40<00:34,  5.50it/s]

In [ ]:
Audio(np.concatenate(pieces), rate=SAMPLE_RATE)

In [ ]:
merged_audio=np.concatenate(pieces)

In [ ]:
import soundfile as sf
sf.write("BARK_TTS.wav", merged_audio, samplerate=16000)

# $ \\ $